Les réseaux de neurones convolutifs ont été utilisés pour reconnaître les chiffres des chèques à la fin des années 90. Il y a eu une base solide pendant tout ce temps, alors pourquoi a-t-on l'impression qu'une explosion s'est produite au cours des dix dernières années ?

Les raisons sont nombreuses, mais la principale d'entre elles est l'augmentation des performances des GPU et leur prix de plus en plus abordable. Conçus à l'origine pour les jeux, les GPU doivent effectuer des millions d'opérations matricielles par seconde afin de restituer tous les polygones du jeu de conduite ou de tir auquel vous jouez sur votre console ou votre PC, opérations pour lesquelles un CPU standard n'est tout simplement pas optimisé.

Un article publié en 2009~[[1]](#1) soulignait que la formation des réseaux de neurones reposait également sur l'exécution d'un grand nombre d'opérations matricielles. Ces cartes graphiques supplémentaires pourraient donc être utilisées pour accélérer la formation et rendre possible, pour la première fois des architectures de réseaux neuronaux plus grandes et plus profondes.

# Démarrer avec PyTorch

In [1]:
!nvidia-smi

Sun Jul 25 18:13:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:2D:00.0  On |                  N/A |
|  0%   48C    P3    48W / 215W |    742MiB /  7981MiB |     30%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

CUDA Version 11.3, on peut donc procéder à l'installation de PyTorch via les instructions trouvable sur leur [site web](https://pytorch.org/).

In [2]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/torch_stable.html


On s'assure ici que CUDA est accesible et crée un tenseur à valeurs aléatoires de taille 2*2

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.rand(2, 2))

True
tensor([[0.7851, 0.0040],
        [0.3255, 0.0973]])


## Tenseurs

Un tenseur est à la fois un conteneur pour les nombres et un ensemble de tuples qui définissent les transformations entre les tenseurs qui produisent de nouveaux tenseurs.

Chaque tenseur a un rang qui correspond à son espace dimensionnel. Un simple scalaire—par exemple, 1—peut être représenté comme un tenseur de rang 0, un vecteur est de rang 1, une matrice $n*n$ est de rang 2, et ainsi de suite. Dans l'exemple précédent, nous avons créé un tenseur de rang 2 avec des valeurs aléatoires en utilisant `torch.rand()`. Nous pouvons également les créer à partir de listes :

In [4]:
x = torch.tensor([[0, 0, 1], [1, 1, 1], [0, 0, 0]])
x

tensor([[0, 0, 1],
        [1, 1, 1],
        [0, 0, 0]])

On peut changer les éléments d'un tenseur en utilisant le système standard d'indexation de Python

In [5]:
x[0][0] = 5
x

tensor([[5, 0, 1],
        [1, 1, 1],
        [0, 0, 0]])

On peut utiliser des fonctions de création spéciales pour générer des types particuliers de tenseurs. En particulier, `ones()` et `zeros()` vont générer des tenseurs remplis de 1 et de 0, respectivement :

In [6]:
torch.zeros(2, 2)

tensor([[0., 0.],
        [0., 0.]])

On peut effectuer des opérations mathématiques standard avec des tenseurs (par exemple, additionner deux tenseurs) :

In [7]:
torch.ones(1, 2) + torch.ones(1, 2)

tensor([[2., 2.]])

Et si vous avez un tenseur de rang 0, vous pouvez en extraire la valeur avec `item()` :

In [8]:
torch.rand(1).item()

0.723783552646637

Les tenseurs peuvent vivre dans le CPU ou sur le GPU et peuvent être copiés entre les dispositifs en utilisant la fonction `to()` :

In [9]:
cpu_tensor = torch.rand(2)
cpu_tensor.device

device(type='cpu')

In [10]:
gpu_tensor = cpu_tensor.to("cuda")
gpu_tensor.device

device(type='cuda', index=0)

## Opérations Tensorielles

Tout d'abord, nous avons souvent besoin de trouver l'élément maximal dans un tenseur ainsi que l'index qui contient la valeur maximale. Ceci peut être fait avec les fonctions `max()` et `argmax()`. Nous pouvons également utiliser `item()` pour extraire une valeur standard Python d'un tenseur à une dimension.

In [11]:
torch.rand(2, 2).max()

tensor(0.6260)

In [12]:
torch.rand(2, 2).max().item()

0.6101565957069397

Parfois, nous souhaitons changer le type d'un tenseur, par exemple en passant d'un **LongTensor** à un **FloatTensor**. Nous pouvons le faire avec `to()` :

In [13]:
long_tensor = torch.tensor([[0, 0, 1], [1, 1, 1], [0, 0, 0]])
long_tensor.type()

'torch.LongTensor'

In [14]:
float_tensor = torch.tensor([[0, 0, 1], [1, 1, 1], [0, 0, 0]]).to(dtype=torch.float32)
float_tensor.type()

'torch.FloatTensor'

La plupart des fonctions qui opèrent sur un tenseur et retournent un tenseur créent un nouveau tenseur pour stocker le résultat. Toutefois, si vous voulez économiser de la mémoire, vérifiez si une fonction *in-place* est définie, elle devrait avoir le même nom que la fonction originale mais avec un underscore (_).

In [15]:
random_tensor = torch.rand(2, 2)
random_tensor.log2()

tensor([[-0.7846, -0.9172],
        [-0.9003, -0.3385]])

In [16]:
random_tensor.log2_()

tensor([[-0.7846, -0.9172],
        [-0.9003, -0.3385]])

Une autre opération courante consiste à remodeler un tenseur. Cela peut souvent se produire parce que la couche de votre réseau de neurones peut nécessiter une forme d'entrée légèrement différente de celle que vous avez actuellement à lui fournir. Par exemple, l'ensemble de données de *Modified National Institute of Standard and Technology* (MNIST) de chiffres manuscrits est une collection de $28*28$ images, mais il est présenté sous forme de tableaux de tenseurs de longueur $1*28*28$ (le 1 initial correspond au nombre de canaux - normalement rouge, vert et bleu - mais comme les chiffres MNIST sont en niveaux de gris, nous n'avons qu'un seul canal). Nous pouvons faire cela avec `view()` ou `reshape()` :

In [17]:
flat_tensor = torch.rand(784)
viewed_tensor = flat_tensor.view(1, 28, 28)
viewed_tensor.shape

torch.Size([1, 28, 28])

In [18]:
reshaped_tensor = flat_tensor.reshape(1, 28, 28)
reshaped_tensor.shape

torch.Size([1, 28, 28])

Maintenant, vous vous demandez peut-être quelle est la différence entre `view()` et `reshape()`. La réponse est que `view()` opère comme une vue sur le tenseur original, donc si les données sous-jacentes sont modifiées, la vue le sera aussi (et vice versa). Cependant, `view()` peut provoquer des erreurs si la vue requise n'est pas [contiguous](https://stackoverflow.com/questions/26998223/what-is-the-difference-between-contiguous-and-non-contiguous-arrays/26999092#26999092) ; c'est-à-dire qu'elle ne partage pas le même bloc de mémoire qu'elle occuperait si un nouveau tenseur de la forme requise était créé de toutes pièces. Si cela se produit, vous devez appeler `tensor.contiguous()` avant de pouvoir utiliser `view()`.

Enfin, vous pouvez avoir besoin de réorganiser les dimensions d'un tenseur. Vous rencontrerez probablement ce problème avec les images, qui sont souvent stockées sous forme de tenseurs \[height, width, channel\], mais PyTorch préfère les traiter dans un \[channel, height, width\]. Vous pouvez utiliser `permute()` pour les traiter de manière assez simple :

In [19]:
hwc_tensor = torch.rand(640, 480, 3)
chw_tensor = hwc_tensor.permute(2, 0, 1)
chw_tensor.shape

torch.Size([3, 640, 480])

# Classification d'Images avec PyTorch

## Défintion du Problème

Ici, nous construisons un classificateur simple qui peut faire la différence entre les poissons et les chats. Nous allons itérer sur la conception et la manière dont nous construisons notre modèle pour le rendre plus précis.

## Challenges Habituels

Premièrement, nous avons besoin de données. Combien de données ? ça dépend. L'idée selon laquelle, pour que toute technique d'apprentissage profond fonctionne, il faut de grandes quantités de données pour entraîner le réseau de neurones n'est pas nécessairement vraie. Cependant, pour l'instant, nous allons nous entraîner à partir de zéro, ce qui nécessite souvent l'accès à une grande quantité de données. Nous avons besoin de beaucoup de photos de poissons et de chats.

Nous pourrions passer du temps à télécharger de nombreuses images à partir d'un moteur de recherche comme Google Image, mais dans ce cas, nous disposons d'un raccourci : une collection standard d'images utilisée pour former les réseaux neuronaux, appelée *ImageNet*. Elle contient plus de 14 millions d'images et 20 000 catégories d'images. C'est la norme à laquelle tous les classificateurs d'images se mesurent.

Comme nous utilisons les données ImageNet, leurs étiquettes ne seront pas très utiles, car elles contiennent trop d'informations pour nous. Une étiquette de chat tabby ou de truite est, pour l'ordinateur, distincte de celle de chat ou de poisson. Nous devrons les réétiqueter.

## PyTorch et le Chargement de Données

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

Le chargement et la conversion des données dans des formats prêts pour l'entraînement peuvent souvent finir par être l'un des domaines de la science des données qui accapare beaucoup trop de notre temps (comme pour ce foutu NER pour WaKED). PyTorch a développé des conventions standard d'interaction avec les données qui rendent le travail assez cohérent, que vous travailliez avec des images, du texte ou de l'audio.

Les deux principales conventions d'interaction avec les données sont les *datasets* et les *data loaders*. Un *datasets* est une classe Python qui nous permet d'accéder aux données que nous fournissons au réseau neuronal. Un *data loaders* est ce qui alimente le réseau en données à partir du *datasets*.

Examinons d'abord le *dataste*. Tout ensemble de données, qu'il comprenne des images, de l'audio, du texte, des paysages en 3D, des informations boursières, ou autre, peut interagir avec PyTorch s'il satisfait à cette classe Python abstraite :

In [21]:
class Dataset(object):
    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

Nous devons implémenter une méthode qui renvoie la taille de notre jeu de données (`len`), et implémenter une méthode qui peut récupérer un élément de notre jeu de données dans une paire (**étiquette**, **tenseur**). Cette méthode est appelée par le *data loader* lorsqu'il introduit des données dans le réseau neuronal pour la formation. Nous devons donc écrire un corps pour `getitem` qui peut prendre l'image et la transformer en un tenseur et le retourner avec le label pour que PyTorch puisse l'utiliser.

## Construire des Données d'Entraînement

In [22]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

Le paquet torchvision comprend une classe appelée ImageFolder qui fait à peu près tout pour nous, à condition que nos images soient dans une structure où chaque répertoire est une étiquette.

In [23]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])

In [24]:
train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)

Les GPU sont conçus pour être rapides dans l'exécution de calculs de taille standard. Mais nous avons probablement un assortiment d'images à de nombreuses résolutions. Pour augmenter nos performances de traitement, nous mettons à l'échelle chaque image entrante à la même résolution de 64*64 via la transformation `Resize(64)`. Nous convertissons ensuite les images en un tenseur, et enfin, nous normalisons le tenseur autour d'un ensemble spécifique de points de moyenne et de déviation standard.

La normalisation est importante car de nombreuses multiplications se produiront lorsque l'entrée passera par les couches du réseau de neurones ; le fait de maintenir les valeurs entrantes entre 0 et 1 empêche les valeurs de devenir trop grandes pendant la phase d'entraînement (connu sous le nom de [*problème d'explosion du gradient*](https://machinelearningmastery.com/exploding-gradients-in-neural-networks/)). 

La moyenne et l'écart-type choisis sont ceux de l'ensemble de données ImageNet dans son ensemble. Vous pourriez les calculer spécifiquement pour ce sous-ensemble de poissons et de chats, mais ces valeurs sont suffisamment décentes. 
Si vous travailliez sur un ensemble de données complètement différent, vous devriez calculer cette moyenne et cet écart, bien que de nombreuses personnes utilisent simplement ces constantes ImageNet et rapportent des résultats acceptables.

## Construire des Données de Test et de Validation

Nous téléchargeons un ensemble de validation, qui est une série d'images de chats et de poissons qui n'apparaissent pas dans l'ensemble d'entraînemnt. À la fin de chaque cycle d'entraînemnt (également appelé *epoch*), nous comparons cet ensemble pour nous assurer que notre réseau ne fait pas d'erreur.

In [25]:
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)

En plus d'un ensemble de validation, nous devons également créer un ensemble de test. Celui-ci est utilisé pour tester le modèle une fois l'entraînement terminé :

In [26]:
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image)

Nous pouvons désormais contruire notre data loader en quelques lignes :

In [27]:
batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

Par défaut, les chargeurs de données de PyTorch sont réglés sur un `batch_size` de 1. Vous voudrez certainement changer cela. Bien que j'ai choisi 64 ici, vous pouvez expérimenter pour voir quelle taille de minibatch vous pouvez utiliser sans épuiser la mémoire de votre GPU. Vous pouvez également expérimenter avec certains des paramètres supplémentaires de PyTorch.

## Création d'un réseau

La création d'un réseau dans PyTorch est une affaire très Pythonique. Nous héritons d'une classe appelée `torch.nn.Network` et remplissons les méthodes `__init__` et `forward` :

In [28]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50,2)
    
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [29]:
simplenet = SimpleNet()

Nous effectuons toute configuration nécessaire dans `init()`, dans ce cas, nous appelons notre constructeur **superclass** et les trois couches entièrement connectées (appelées `Linear` dans PyTorch, par opposition à `Dense` dans Keras). La méthode `forward_()` décrit comment les données circulent à travers le réseau à la fois pour l'entraînement et pour faire des prédictions (inférence). Tout d'abord, nous devons convertir le tenseur 3D ($x$ et $y$ plus les informations de couleur à trois canaux - rouge, vert, bleu -) en un tenseur 1D afin qu'il puisse être introduit dans la première couche `Linear`, et nous le faisons en utilisant `view()`. A partir de là, vous pouvez voir que nous appliquons les fonctions d'activation des couches dans l'ordre, pour finalement retourner la sortie `softmax` et nous donner notre prédiction pour cette image.

Les nombres dans les couches cachées sont quelque peu arbitraires, à l'exception de la sortie de la dernière couche, qui est 2, correspondant à nos deux classes. En général, vous voulez que les données de vos couches soient compressées au fur et à mesure qu'elles descendent dans la pile. Si une couche va, disons, de 50 entrées à 100 sorties, alors le réseau pourrait apprendre en passant simplement les 50 connexions à 50 des 100 sorties et considérer que son travail est terminé. En réduisant la taille de la sortie par rapport à l'entrée avec moins de ressources, ce qui signifie, espérons-le, qu'il extrait certaines caractéristiques des images qui sont importantes pour le problème que nous essayons de résoudre ; par exemple, apprendre à repérer une nageoire ou une queue.

Nous avons une prédiction, et nous pouvons la comparer avec l'étiquette réelle de l'image originale pour voir si la prédiction était correcte. Mais nous avons besoin d'un moyen de permettre à PyTorch de quantifier non seulement si une prédiction est juste ou fausse, mais aussi à quel point elle est juste ou fausse. Ceci est géré par une fonction de perte (*loss function*).

## Fonctions de Perte (*Loss Functions*)

PyTorch est livré avec une collection complète qui couvre la plupart des applications que vous êtes susceptible de rencontrer, et vous pouvez bien sûr écrire vos propres applications si vous avez un domaine très personnalisé. Dans notre cas, nous allons utiliser une fonction de perte intégrée appelée `CrossEntropyLoss` qui est recommandée pour les tâches de catégorisation multiclasse comme celle que nous effectuons ici. Une autre fonction de perte que vous êtes susceptible de rencontrer est `MSELoss`, qui est la perte quadratique moyenne standard que vous pourriez utiliser lorsque vous faites une prédiction numérique.

Une chose à laquelle il faut faire attention avec `CrossEntropyLoss` est qu'il incorpore également `softmax()` dans le cadre de ses opérations, ainsi notre méthode `forward_()` devient la suivante :

```python
%%add_to SimpleNet

def forward(self, x):
    x = x.view(-1, 12288)
    x = torch.nn.functional.relu(self.fc1(x))
    x = torch.nn.functional.relu(self.fc2(x))
    x = self.fc3(x)
    return x
```

## Optimisation

Pour le problème des minima locaux, nous apportons une légère modification au fait que nous prenons tous les gradients possibles et indiquons des gradients aléatoires pendant un batch. Connue sous le nom de [*stochastic gradient descent* (SGD)](https://www.geeksforgeeks.org/ml-stochastic-gradient-descent-sgd/) [[2]](#2), il s'agit de l'approche traditionnelle de l'optimisation des réseaux neuronaux et autres techniques d'apprentissage automatique. Mais d'autres optimiseurs sont disponibles, et en fait pour l'apprentissage profond, préférables. PyTorch est livré avec SGD et d'autres optimiseurs comme [AdaGrad](https://www.geeksforgeeks.org/intuition-behind-adagrad-optimizer/) et [RMSProp](https://towardsdatascience.com/understanding-rmsprop-faster-neural-network-learning-62e116fcf29a), ainsi qu'[Adam](https://www.geeksforgeeks.org/intuition-of-adam-optimizer/) [[3]](#3).

L'une des principales améliorations apportées par **Adam** (tout comme **RMSProp** et **AdaGrad**) est qu'il utilise un taux d'apprentissage par paramètre, et adapte ce taux d'apprentissage en fonction du taux de changement de ces paramètres. Il conserve une liste exponentiellement décroissante de gradients et du carré de ces gradients et les utilise pour mettre à l'échelle le taux d'apprentissage global avec lequel **Adam** travaille. Il a été démontré empiriquement que **Adam** surpasse la plupart des autres optimiseurs dans les réseaux d'apprentissage profond, mais vous pouvez remplacer Adam par **SGD** ou **RMSProp** ou un autre optimiseur pour voir si l'utilisation d'une technique différente permet un apprentissage plus rapide et plus efficace pour votre application particulière.

La création d'un optimiseur basé sur **Adam** est simple. Nous appelons `torch.optim.Adam()` et passons les poids du réseau qu'il va mettre à jour (obtenus via `simmplenet.parameters()`) et notre taux d'apprentissage de 0.001 :

In [30]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

## Entraînement

```python
for epoch in range(epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        imput, target = batch
        output = model(input)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
```

Nous prenons un batch de notre ensemble d'entraînement à chaque itération de la boucle, ce qui est géré par notre data loader. Nous les soumettons ensuite à notre modèle et calculons la perte à partir de la sortie attendue. Pour calculer les gradients, nous appelons la méthode `backward()` sur le modèle. La méthode `optimizer.step()` utilise ensuite ces gradients pour effectuer l'ajustement des poids dont nous avons parlé dans la section précédente.

Mais à quoi sert cet appel `zero_grad()` ? Il s'avère que les gradients calculés s'accumulent par défaut, ce qui signifie que si nous ne mettons pas à zéro les gradients à la fin de l'itération du batch, le batch suivant devra gérer le gradient de ce batch ainsi que le sien, et le batch suivant devra gérer les deux précédents, et ainsi de suite. Cela n'est pas utile, car nous voulons examiner uniquement les gradients du batch actuel pour notre optimisation à chaque itération. Nous utilisons `zero_grad()` pour nous assurer qu'ils sont remis à zéro une fois que nous avons terminé notre boucle.

## Fonctionnement sur GPU

In [31]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

Ici, nous copions le modèle sur le GPU si PyTorch signale qu'un GPU est disponible, ou sinon nous gardons le modèle sur le CPU. 

## Mise en Commun

In [32]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [33]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=5, device=device)

Epoch: 1, Training Loss: 2.12, Validation Loss: 7.59, accuracy = 0.31
Epoch: 2, Training Loss: 3.60, Validation Loss: 1.09, accuracy = 0.71
Epoch: 3, Training Loss: 0.89, Validation Loss: 1.07, accuracy = 0.59
Epoch: 4, Training Loss: 0.74, Validation Loss: 0.58, accuracy = 0.78
Epoch: 5, Training Loss: 0.41, Validation Loss: 0.70, accuracy = 0.72


## Prédictions

Voici un petit bout de code Python qui chargera une image depuis le système de fichiers et dira si notre réseau prédit un chat ou poisson.

In [34]:
labels = ['cat','fish']

img = Image.open("./val/fish/100_1422.JPG") 
img = img_transforms(img).to(device)
img = torch.unsqueeze(img, 0)

simplenet.eval()
prediction = F.softmax(simplenet(img), dim=1)
prediction = prediction.argmax()
print(labels[prediction])

fish


Comme notre réseau utilise des lots, il s'attend en fait à un tenseur 4D, la première dimension représentant les différentes images d'un lot. Nous n'avons pas de lot, mais nous pouvons créer un lot de longueur 1 en utilisant `unsqueeze(0)`, qui ajoute une nouvelle dimension à l'avant de notre tenseur.

## Sauvegarder le Modèle

Si vous êtes satisfait des performances d'un modèle ou si vous avez besoin de vous arrêter, vous pouvez sauvegarder l'état actuel d'un modèle au format *pickle* de Python en utilisant la méthode `torch.save()`. Inversement, vous pouvez charger une interation précédemment sauvegardée d'un modèle en utilisant la méthode `torch.load()`.

In [35]:
torch.save(simplenet, "/tmp/simplenet") 
simplenet = torch.load("/tmp/simplenet")

Cela permet de stocker les paramètres et la structure du modèle dans un fichier. Cela peut poser un problème si vous modifiez la structure du modèle ultérieurement. Pour cette raison, il est plus courant de sauvegarder le `state_dict` d'un modèle à la place. C'est un `dict` Python standard qui contient les cartes pour les paramètres de chaque couche du modèle.

In [36]:
torch.save(simplenet.state_dict(), "/tmp/simplenet")    
simplenet = SimpleNet()
simplenet_state_dict = torch.load("/tmp/simplenet")
simplenet.load_state_dict(simplenet_state_dict)

<All keys matched successfully>

# Réseaux Neuronaux Convolutifs

Avec des réseaux entièrement connectés, si vous essayez d'ajouter des couches supplémentaires ou d'augmenter considérablement le nombre de paramètres, vous allez certainement manquer de mémoire sur votre GPU. De plus, l'entraînement jusqu'à une précision décente prendrait un certain temps.

Il est vrai qu'un réseau entièrement connecté ou (*feed-forward*) peut fonctionner comme un approximateur universel, mais la théorie ne dit pas combien de temps il vous faudra pour l'entraîner à devenir cette approximation de la fonction que vous recherchez vraiment. Mais on peut faire mieux, surtout avec les images.

## Premier Modèle Convolutif

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

In [38]:
class CNNNet(nn.Module):

    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

La première chose à remarquer est l'utilisation de `nn.Sequential()`. Cela nous permet de créer une chaîne de couches (*layers*). Lorsque nous utilisons une de ces chaînes dans `forward()`, l'entrée passe par chacune des couches en succession. Vous pouvez utiliser ceci pour décomposer votre modèle en arrangements plus logiques. Dans ce réseau, nous avons deux chaînes : le bloc `features` et le `classifier`. Jetons un coup d'oeil aux nouvelles couches que nous introduisons, en commençant par `Conv2d`.

## Convolutions

La couche `Conv2d` est une convolution 2D. Si nous avons une image en niveaux de gris, elle consiste en un tableau de $x$ pixels de large et $y$ pixels de haut, dont chaque entrée a une valeur qui indique si elle est noire ou blanche ou quelque part entre les deux (nous supposons une image de 8 bits, donc chaque valeur peut varier de 0 à 255).

![](https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_02_17A-ConvolutionalNeuralNetworks-WHITEBG.png)

Une couche convolutive comportera plusieurs de ces filtres, dont les valeurs sont remplies par l'apprentissage du réseau, et tous les filtres de la couche partagent les mêmes valeurs de biais. Revenons à la façon dont nous invoquons la couche `Conv2d` et voyons certaines des autres options que nous pouvons définir :

```python
nn.Conv2d(
    in_channels,
    out_channels,
    kernel_size,
    stride,
    padding
)
```

Le `in_channels` est le nombre de canaux d'entrée que nous allons recevoir dans la couche. Au début du réseau, nous avons une image RGB comme entrée, donc le nombre de canaux d'entrée est de trois. `out_channels` est, sans surprise, le nombre de canaux de sortie, qui correspond au nombre de filtres dans notre couche conv. Ensuite, il y a `kernel_size`, qui décrit la hauteur et la largeur de notre filtre. Cela peut être un simple scalaire spécifiant un carré (par exemple, dans la première couche conv, nous mettons en place un filtre 11\*11), ou vous pouvez utiliser un tuple (comme (3, 5) pour un filtre 3\*5).

`stride` indique combien de pas sur l'entrée nous nous déplaçons lorsque nous ajustons le filtre à une nouvelle position. Dans notre exemple, nous nous retrouvons avec un stride de 2, ce qui a pour effet de créer une *features map* qui a la moitié de la taille de l'entrée. Mais nous aurions également pu nous déplacer avec un stride de 1, ce qui nous aurait donné une *features map* de 4\*4, de la même taille que l'entrée. Nous pouvons également passer un tuple $(a, b)$ qui nous permettrait de déplacer $a$ vers le haut et $b$ vers le bas à chaque étape.

Si vous ne définissez pas `padding`, tous les cas limites que PyTorch rencontre dans les dernières colonnes de l'entrée sont simplement jetés. C'est à vous de définir le padding de manière appropriée. Comme avec `stride` et `kernel_size`, vous pouvez aussi passer un tuple pour le remplissage **height\*weight** au lieu d'un nombre unique qui remplit les deux sens de la même manière.

### Informations complémentaires

* Pour une matrice d'entre $M_{m, n}$, $m, n \in \mathbb{N}$ que l'on convolue à une matrice $F_{f, g}$, $f, g \in \mathbb{N}$, le tout à l'aide d'un padding $p$, on obtient une matrice de sortie de dimensions $m + 2p - f + 1 \times n + 2p - g + 1$
* Ainsi, afin d'effectuer une convolution similaire (où la sortie est de même dimension que l'entrée), on prend $p$ tel que $p = \frac{f - 1}{2}$ (pour le nombre de colonnes ici).
* Par convention, en vision par ordinateur, les filtres de convolutions sont presques toujours de dimensions impairs, d'une part pour simplifier les calculs et d'autre part afin d'avoir un pixel central auqeul se réferrer sur le filtre.
* Si on utilise désormais un stride $s$ en plus du padding, on se retrouve avec une matrice en sortie de dimension $\lfloor \frac{m + 2p - f}{s}\rfloor \times \lfloor\frac{n + 2p - g}{s}\rfloor$ (pour le nombre de colonnes ici).
* Ainsi, afin d'effectuer une convolution similaire lorsqu'un stride est effectué, on prend $p$ tel que $p = \lfloor \frac{m(s - 1) + f}{2}\rfloor$

* [Padding](https://www.youtube.com/watch?v=smHa2442Ah4)
* [Stride](https://www.youtube.com/watch?v=tQYZaDn_kSg)

## Dropout

Et si nous n'entraînions pas un groupe aléatoire de nœuds du réseau au cours d'un cycle d'entraînement ? Comme ils ne seront pas mis à jour, ils n'auront pas la possibilité de s'overfit, et comme c'est aléatoire, chaque cycle de formation ignorera une sélection différente des données d'entrée, ce qui devrait favoriser la généralisation.

Par défaut, les couches `Dropout` de notre exemple CNN sont initialisées avec $0.5$, ce qui signifie que 50% du tenseur d'entrée est aléatoirement mis à zéro. Si vous voulez changer cela à 20%, ajoutez le paramètre `p` à l'appel d'initialisation : `Dropout(p=0.2)`.

Attention cependant à ce que le `Dropout` ne soit effectué que lors de l'inférence (ce qui est le cas par défaut avec PyTorche)

![](https://blogs.gartner.com/paul-debeasi/files/2019/01/Train-versus-Inference.png)

# Transfert Learning et Autres Techniques

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

Nous n'avons pas affaire à une architecture qui a été initialisée avec des paramètres aléatoires, comme nous l'avons fait dans le passé. Notre modèle ResNet pré-entraîné contient un tas d'informations encodées pour la reconnaissance et la classification d'images, alors pourquoi tenter de le réentraîner ? Au lieu de cela, nous affinons le réseau. Nous modifions légèrement l'architecture pour inclure un nouveau bloc de réseau à la fin, remplaçant les couches linéaires standard de 1000 catégories qui effectuent normalement la classification ImeNet. Nous gelons ensuite toutes les couches ResNet existantes, et lorsque nous nous entraînons, nous ne mettons à jour que les paramètres de nos couches, mais nous prenons toujours l'activation de nos couches gelées. Cela nous permet d'entraîner rapidement nos nouvelles couches tout en préservant les informations que les couches pré-entraînées contiennent déjà.


Tout d'abord, créons un modèle ResNet-50 pré-entraîné :

In [40]:
transfer_model = models.resnet50(pretrained=True)

Ensuite, nous devons geler les couches. La façon dont nous faisons cela est simple : nous les empêchons d'accumuler des gradients en utilisant `requires_grad()`. Nous devons faire cela pour chaque paramètre du réseau, mais heureusement, PyTorch fournit une méthode `parameters()` qui rend cela plutôt facile.

Vous ne voudrez peut-être pas geler les couches `BatchNorm` d'un modèle, car elles seront entraînées à se rapprocher de la moyenne et de l'écart-type de l'ensemble de données sur lequel le modèle a été entraîné à l'origine, et non de l'ensemble de données sur lequel vous souhaitez effectuer un réglage fin. Une partie du signal de vos données peut finir par être perdue lorsque `BatchNorm` corrige vos entrées. Vous pouvez regarder la structure du modèle et geler seulement les couches qui ne sont pas `BatchNorm` comme ceci :

In [41]:
for name, param in transfer_model.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

In [42]:
transfer_model.fc = nn.Sequential(nn.Linear(transfer_model.fc.in_features,500),
nn.ReLU(),                                 
nn.Dropout(), nn.Linear(500,2))

Ensuite, nous devons remplacer le bloc de classification final par un nouveau bloc que nous entraînerons à détecter les chats ou les poissons. Dans cet exemple, nous le remplaçons par un couple de couches `Linear`, un `ReLU`, et un `Dropout`, mais vous pourriez avoir des couches CNN supplémentaires ici aussi. Heureusement, la définition de l'implémentation de ResNet par PyTorch stocke le bloc de classification finale comme une variable d'instance, `fc`, donc tout ce que nous avons à faire est de la remplacer par notre nouvelle structure (les autres modèles fournis avec PyTorch utilisent soit `fc` soit `classifier`).

Dans le code précédent, nous profitons de la variable `in_features` qui nous permet de saisir le nombre d'activations entrant dans une couche (2048 dans ce cas). Vous pouvez également utiliser `out_features` pour découvrir les activations qui sortent.

## Trouver le *Learning rate*

Un article de Leslie Smith, chercheuse scientifique au Laboratoire de recherche navale des États-Unis, contient une approche permettant de trouver un taux d'apprentissage approprié [[4]](#4).Au cours d'une époque, on commence avec un taux d'apprentissage faible et on augmente le taux d'apprentissage à chaque mini-batch, pour obtenir un taux élevé à la fin de l'époque. Calculez la perte pour chaque taux et ensuite, en regardant un graphique, choisissez le taux d'apprentissage qui donne la plus grande baisse (la pente la plus élevée).

In [43]:
def find_lr(model, loss_fn, optimizer, train_loader, init_value=1e-8, final_value=10.0, device="cpu"):
    number_in_epoch = len(train_loader) - 1
    update_step = (final_value / init_value) ** (1 / number_in_epoch)
    lr = init_value
    optimizer.param_groups[0]["lr"] = lr
    best_loss = 0.0
    batch_num = 0
    losses = []
    log_lrs = []
    for data in train_loader:
        batch_num += 1
        inputs, targets = data
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        # Crash out if loss explodes

        if batch_num > 1 and loss > 4 * best_loss:
            if(len(log_lrs) > 20):
                return log_lrs[10:-5], losses[10:-5]
            else:
                return log_lrs, losses

        # Record the best loss

        if loss < best_loss or batch_num == 1:
            best_loss = loss

        # Store the values
        losses.append(loss.item())
        log_lrs.append((lr))

        # Do the backward pass and optimize

        loss.backward()
        optimizer.step()

        # Update the lr for the next step and store

        lr *= update_step
        optimizer.param_groups[0]["lr"] = lr
    if(len(log_lrs) > 20):
        return log_lrs[10:-5], losses[10:-5]
    else:
        return log_lrs, losses

Nous passons nos entrées par le modèle et ensuite nous obtenons la perte du batch. Nous enregistrons la meilleure perte jusqu'à présent et nous la comparons à la nouvelle perte. Si la nouvelle perte est plus de quatre fois supérieure à `best_loss`, nous sortons de la fonction en retournant ce que nous avons jusqu'à présent (car la perte tend probablement vers l'infini). Sinon, nous continuons à ajouter la perte et les logs du taux d'apprentissage actuel, et nous mettons à jour le taux d'apprentissage avec l'étape suivante sur le chemin vers le taux maximal à la fin de la boucle. Le graphique peut alors être affiché en utilisant la fonction `matplotlib plt` :

```python
(lrs, losses) = find_lr(transfer_model, torch.nn.CrossEntropyLoss(),optimizer, train_data_loader,device=device)
plt.plot(lrs, losses)

plt.xscale("log")
plt.xlabel("Learning rate")
plt.ylabel("Loss")
plt.show()
```

Nous renvoyons des tranches des journaux et des pertes `lr`. Nous faisons cela simplement parce que les premiers bits d'apprentissage et les derniers (surtout si le taux d'apprentissage devient très grand assez rapidement) ont tendance à ne pas nous donner beaucoup d'informations.

Enfin, rappelez-vous qu'étant donné que cette fonction entraîne le modèle et modifie le taux d'apprentissage de l'optimiseur, vous devez sauvegarder et recharger votre modèle avant de le faire pour revenir à l'état dans lequel il était avant d'appeler `find_lr()` et aussi réinitialiser l'optimiseur que vous avez choisi avec le taux d'apprentissage choisi.

## Taux d'Apprentissage Différentiel

Lors des entraînements jusqu'à présent, nous avons appliqué un seul taux d'apprentissage à l'ensemble du modèle. Mais lorsqu'il s'agit de transfert learning, nous pouvons normalement obtenir une meilleure précision si nous essayons quelque chose de différent : former différents groupes de couches à différents taux. Plus tôt, nous avons gelé toutes les couches pré-entraînées de notre modèle et entraîné uniquement notre nouveau classificateur, mais nous pouvons souhaiter affiner certaines des couches du modèle ResNet que nous utilisons, par exemple. Peut-être qu'en ajoutant un peu d'entraînement aux couches précédant notre classificateur, notre modèle sera un peu plus précis. Mais ces couches précédentes ont déjà été entraînées sur le jeu de données ImageNet, peut-être n'ont-elles besoin que d'un peu d'entraînement par rapport à nos nouvelles couches ? PyTorch offre un moyen simple d'y parvenir. Modifions notre optimiseur pour le modèle ResNet-50 :

```python
optimizer = optimizer.Adam([
    {"params": transfer_model.layer4.parameters(), "lr": found_lr / 3},
    {"params": transfer_model.layer3.parameters(), "lr": found_lr / 9}
], lr=found_lr)
```

Nous avons gelé toutes ces couches pré-entraînées. C'est très bien de leur donner un taux d'apprentissage différent, mais pour l'instant, l'entraînement du modèle ne les touchera pas du tout car elles n'accumulent pas de gradients.

```python
unfreeze_layers = [transfer_model.layer3, transfer_model.layer4]
for layer in unfreeze_layers:
    for param in layer.parameters():
        param.requires_grad = True
```

## Augmentation de Données

## Torchvision Transforms

## Espaces de Couleur et Trasformations Lambda

## Classes de Transformation Personnalisées

## Start Small and Get Bigger!

## Ensembles

L'assemblage (*ensembling*) est une technique assez courante dans les méthodes d'apprentissage automatique plus traditionnelles, et elle fonctionne plutôt bien dans l'apprentissage profond également. L'idée est d'obtenir une prédiction à partir d'une série de modèles, et de combiner ces prédictions pour produire une réponse finale. Étant donné que les différents modèles ont des forces différentes dans différents domaines, on peut espérer que la combinaison de toutes leurs prédictions produira un résultat plus précis qu'un modèle seul.

Pour commencer à utiliser les ensembles, il suffit de faire la moyenne des prédictions :

```python
models_ensemble = [models.resnet50().to(device), models.resnet50().to(device)]
predictions = [F.softmax(m(torch.rand(1,3,224,244).to(device))) for m in models_ensemble] 
avg_prediction = torch.stack(predictions).mean(0).argmax()
```

```python
avg_prediction
```

```python
torch.stack(predictions)
```

La méthode `stack` concatène le tableau de tenseurs ensemble, donc si nous travaillons sur le problème chat/poisson et que nous avons quatre modèles dans notre ensemble, nous nous retrouverons avec un tenseur de 4 \times 2$ construit à partir des quatre tenseurs de 1 \times 2$. Et `mean` fait ce que vous attendez, bien que nous devions passer dans une dimension de 0 pour s'assurer qu'il prend une moyenne sur la première dimension au lieu de simplement additionner tous les éléments du tenseur et produire une sortie scalaire. Enfin, `argmax` sélectionne l'indice du tenseur avec l'élément le plus élevé.

Il est facile d'imaginer des approches plus complexes. On pourrait peut-être ajouter des pondérations à la prédiction de chaque modèle individuel, et ajuster ces pondérations si un modèle obtient une réponse juste ou fausse.

# Classification de textes

## Réseau de Neurones Récurrent (*Recurrent Neural Network, RNN*)

Si nous examinons la façon dont nous avons utilisé notre architecture basée sur CNN jusqu'à présent, nous pouvons constater qu'elle a toujours travaillé sur un instant précis de temps. Mais considérez ces deux fragments :
> The cat sat on the mat.

> She got up and impatiently climbed on the chair, meowing for food.

Supposons que vous donniez ces deux phrases, l'une après l'autre, à un CNN et que vous lui demandiez : "Où est le chat ?". Vous auriez un problème, car le réseau n'a aucun concept de mémoire.

Les réseaux neuronaux récurrents (RNN) répondent à ce problème en dotant le réseau neuronal d'une mémoire via un état caché. Nous ajoutons une entrée à un pas de temps de $t$, et nous obtenons un état de sortie caché de $ht$, et la sortie est également réinjectée dans le RNN pour le pas de temps suivant.

## Réseaux de Mémoire à Long Terme (*Long Short-Term memory Networks, LSTM*)

In [ ]:
import spacy
import torchtext
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy import data

In [ ]:
device = "cuda"

En pratique, les RNN étaient et sont particulièrement sensibles au *vanishing gradient problem*, ou au scénario potentiellement pire du *exploding gradient*, où votre erreur tend vers l'infini. Ni l'un ni l'autre n'est bon, de sorte que les RNN ne pouvaient pas être utilisés pour la plupart des problèmes pour lesquels ils étaient considérés comme adaptés.

## *Embedding*

Quant à l'utilisation des embeddings dans PyTorch, c'est très simple :
```python
embed = nn.Embedding(vocab_size, dimension_size)
```
Il contiendra un tenseur de `vocab_size` $\times$ `dimension_size` initialisé aléatoirement. Chaque mot de votre vocabulaire indexe une entrée qui est un vecteur de `dimension_size`, donc si nous revenons à notre chat et ses aventures épiques sur le tapis, nous aurions quelque chose comme ceci :
```python
cat_mat_embed = nn.Embeding(5, 2)
cat_tensor = embed.Tensor([1])
cat_mat_embed.forward(cat_tensor)
```
Nous créons notre *embedding*, un tenseur qui contient la position du mot chat dans notre vocabulaire, et nous le passons à la méthode `forward()` des couches. Cela nous donne notre *embedding* aléatoire. le résultat indique également que nous avons une fonction de gradient que nous pouvons utiliser pour la mise à jour des paramètres après l'avoir combinée avec une fonction de perte.

## Définir les Champs

**torchtext** adopte une approche directe pour générer des ensembles de données : vous lui dites ce que vous voulez, et il traitera le CSV (ou JSON) brut pour vous. Pour ce faire, vous devez d'abord définir des *fields*. La classe `Field` possède un nombre considérable de paramètres qui peuvent lui être assignés :

Paramètre|Description|Défaut|
---------|-----------|------|
`sequential`|Si le champ représente des données séquentielles (c'est-à-dire du texte). S'il a pour valeur `False`, aucune tokenisation n'est appliquée.|`True`|
`use_vocab`|Si l'on veut inclure un objet `Vocab`. S'il a pour valeur `False`, le champ doit contenir des données numériques.|`True`|
`init_token`|Un jeton qui sera ajouté au début de ce champ pour indiquer le début des données.|`None`|
`eos_token`|Jeton de fin de phrase (*end-of-sentence*), ajouté à la fin de chaque séquence.|`None`|
`fix_length`|Si elle est définie comme un nombre entier, toutes les entrées seront complétées à cette longueur. Si elle n'est pas définie, la longueur des séquences sera flexible.|`None`|
`dtype`|Le type du lot de tenseur.|`torch.long`|
`lower`|Converti la séquence en minuscule|`False`|
`tokenize`|La fonction qui va effectuer la tokenisation de la séquence. Si elle est définie sur `spacy`, le tokenizer spaCy sera utilisé.|`string`,`split`|
`pad_token`|Le jeton qui sera utilisé pour le *padding*|`<pad>`|
`unk_token`|Le jeton qui sera utilisé pour représenter les mots qui ne sont pas présents dans le `Vocab dict`.|`<unk>`|
`pad_first`|*Pad* le début de la séquence.|`False`|
`truncate_first`|Tronquer au début de la séquence (si nécessaire).|`False`|

Par exemple si nous nous interessons uniquement à des tweets ainsi qu'à leur label pour e l'analyse de sentiment, nous utilisons l'objet `Field` :

In [ ]:
LABEL = data.LabelField()
TWEET = data.Field('spacy', tokenizer_language='en_core_web_sm', lower=True)

Nous définissons le `LABEL` comme `LabelFIeld`, qui est une sous-classe de `Field` qui met `sequential` à `False`. `TWEET` est un objet `Field` standard, pour lequel nous avons décidé d'utiliser le tokenizer spaCy et de convertir tout le texte en minuscules, mais sinon nous utilisons les valeurs par défaut listées dans le tableau précédent. Si, lors de l'exécution de cet exemple, l'étape de construction du vocabulaire prend beaucoup de temps, essayez de supprimer le paramètre `tokenize` et recommencez. Ceci utilisera le paramètre par défaut qui consiste à simplement séparer sur les espaces, ce qui accélérera considérablement l'étape de tokenisation, bien que le vocabulaire créé ne sera pas aussi bon que celui créé par spaCy.

Après avoir défini ces champs, nous devons maintenant produire une liste qui les met en correspondance avec une liste des colonnes qui se trouvent dans un CSV.

In [ ]:
fields = [('score',None), ('id',None), ('date',None), ('query',None),
          ('name',None), ('tweet', TWEET), ('category',None), ('label',LABEL)]

Armés de nos champs déclarés, nous utilisons maintenant `TabularDataset` pour appliquer cette définition au CSV.

In [ ]:
twitterDataset = data.dataset.TabularDataset(
        path="train-processed-sample.csv", 
        format="CSV", 
        fields=fields,
        skip_header=False)

Cela peut prendre un certain temps, surtout avec l'analyseur syntaxique spaCy. Enfin, nous pouvons séparer les ensembles d'entraînement, de test et de validation en utilisant la méthode `split()`.

In [ ]:
(train, test, valid) = twitterDataset.split(split_ratio=[0.6,0.2,0.2],
                                            stratified=True, strata_field='label')

(len(train),len(test),len(valid))

## Construire un Vocabulaire

Traditionnellement, à ce stade, nous construisons un *one-hot encoding* de chaque mot présent dans l'ensemble de données. Heureusement, **torchtext** le fera pour nous, et permettra également de passer un paramètre `max_size` pour limiter le vocabulaire aux mots les plus courants. Ceci est normalement fait pour éviter la construction d'un modèle énorme et gourmand en mémoire. Nous ne voulons pas que nos GPUs soient surchargés, après tout. Limitons le vocabulaire à un maximum de 20 000 mots dans notre ensemble d'apprentissage.

In [ ]:
vocab_size = 20000
TWEET.build_vocab(train, max_size = vocab_size)
LABEL.build_vocab(train)
TWEET.vocab.freqs.most_common(10)

Nous avons presque terminé avec nos jeux de données. Il ne nous reste plus qu'à créer un chargeur de données pour alimenter notre boucle d'entraînement. `torchtext` fournit la méthode `BucketIterator` qui produira ce qu'il appelle `Batch`, qui est presque, mais pas tout à fait, comme le chargeur de données que nous avons utilisé sur les images.

In [ ]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test),
    batch_size = 32,
    device = device,
    sort_key = lambda x: len(x.tweet),
    sort_within_batch = False)

## Création du Modèle

In [ ]:
class OurFirstLSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(OurFirstLSTM, self).__init__()
    
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size=embedding_dim,  
                hidden_size=hidden_size, num_layers=1)
        self.predictor = nn.Linear(hidden_size, 2)

    def forward(self, seq):
        output, (hidden,_) = self.encoder(self.embedding(seq))
        preds = self.predictor(hidden.squeeze(0))
        return preds

model = OurFirstLSTM(100,300, 20002)
model.to(device)

Tout ce que nous faisons dans ce modèle est de créer trois couches. Tout d'abord, les mots de nos tweets sont poussés dans une couche `Embedding`, que nous avons établie comme un vecteur d'intégration à 300 dimensions. C'est ensuite alimenté dans un `LSTM` avec 100 caractéristiques cachées. Enfin, la sortie du LSTM (l'état caché final après le traitement du tweet entrant) est poussée à travers une couche standard entièrement connectée avec trois sorties pour correspondre à nos trois classes possibles (négatif, positif ou neutre).

## Mise à Jour de la Boucle d'Entraînement

En raison de certaines bizarreries du `torchtext`, nous devons écrire une boucle d'entraînement légèrement modifiée. D'abord, nous créons un optimiseur et une fonction de perte. Parce que nous avons reçu trois classes potentielles pour chaque tweet, nous utilisons `CrossEntropyLoss()` comme fonction de perte.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()

def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs+1):
     
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * batch.tweet.size(0)
        training_loss /= len(train_iterator)
 
        
        model.eval()
        for batch_idx,batch in enumerate(valid_iterator):
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            valid_loss += loss.data.item() * batch.tweet.size(0)
 
        valid_loss /= len(valid_iterator)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))

La principale chose dont il faut être conscient dans cette nouvelle boucle d'apprentissage est que nous devons référencer `batch.tweet` et `batch.label` pour obtenir les champs particuliers qui nous intéressent ; ils ne tombent pas aussi bien de l'énumérateur qu'ils le font dans `torchvision`.

In [ ]:
train(5, model, optimizer, criterion, train_iterator, valid_iterator)

## Prédictions

In [ ]:
def classify_tweet(tweet):
    categories = {0: "Negative", 1:"Positive"}
    processed = TWEET.process([TWEET.preprocess(tweet)])
    processed = processed.to(device)
    model.eval()
    return categories[model(processed).argmax().item()]

## Augmentation de Données

En 2019, dans *EDA: Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks* [[5]](#5) trois techniques d'augmentation de données sont suggérées : l'insertion aléatoire, la permutation aléatoire et la suppression aléatoire.

Une technique d'insertion aléatoire examine une phrase, puis insère de manière aléatoire des synonymes de mots nonsop existants dans la phrase $n$ fois. En supposant que vous avez un moyen d'obtenir un synonyme d'un mot et un moyen d'éliminer les mots de fin, montrés, mais non implémentés, dans cette fonction via `get_synonyms()` et `get_stopwords()`, une implémentation de ceci serait comme suit :

```python
def random_insertion(sentence,n):
    words = remove_stopwords(sentence)
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym)
    return sentence



def random_swap(sentence, n=5):
    length = range(len(sentence))
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1]
    return sentence

def random_deletion(words, p=0.5):
    if len(words) == 1:
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    if len(remaining) == 0:
        return [random.choice(words)]
    else:
        return remaining
```

### Suppression Aléatoire

# Références

<a id="1">[1]</a> 
Rajat Raina et al. (2009). 
"Large-Scale Deep Unsupervised Learning Using Graphics Processors". 
*Proceedings of the 26th Annual International Conference on Machine Learning*, 873–880.
https://doi.org/10.1145/1553374.1553486

<a id="2">[2]</a> 
Sebastian Ruder (2016). 
"An overview of gradient descent optimization algorithms". 
*CoRR*, abs/1609.04747.
https://arxiv.org/abs/1609.04747

<a id="3">[3]</a> 
Diederik P.Kingama, Jimmy Ba (2014). 
"Adam: A Method for Stochastic Optimization". 
*3rd International Conference on Learning Representations, ICLR 2015*.
https://arxiv.org/abs/1412.6980

<a id="4">[4]</a> 
Leslie Smith (2015). 
"Cyclical Learning Rates for Training Neural Networks".
https://arxiv.org/abs/1506.01186

<a id="5">[5]</a> 
Jason W. Wei, Kai Zou (2019). 
"EDA: Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks".
*CoRR*, abs/1901.11196.
http://arxiv.org/abs/1901.11196